In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/creditcardfraud/creditcard.csv


In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import keras
import os
import matplotlib.pyplot as plt
import seaborn as sn
import itertools
from collections import Counter
np.random.seed(2)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, f1_score, recall_score
from imblearn.over_sampling import SMOTE
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

In [3]:
data = pd.read_csv('../input/creditcardfraud/creditcard.csv')

In [4]:
# scaling amount and time
scaler = StandardScaler()
data['scaled_amount'] = scaler.fit_transform(data['Amount'].values.reshape(-1, 1))

In [5]:
#split data
data = data.drop(['Amount', 'Time'], axis = 1)
y = data['Class']
X = data.drop(['Class'], axis = 1)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [7]:
#convert data for fitting to neural networks 
train_identity = X_train.index
test_identity = X_test.index
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [8]:
# Create deep learning model
model = Sequential()
#add input layer
model.add(Dense(input_dim = 29, units = 16, activation = 'relu'))
#add 2nd hidden layer
model.add(Dense(units = 24, activation = 'relu'))
#add dropout layer
model.add(Dropout(0.5))
#add 3rd hidden layer
model.add(Dense(units = 20, activation = 'relu'))
#add 4th hidden layer
model.add(Dense(units = 24, activation = 'relu'))
#add ouptut layer
model.add(Dense(units = 1, activation = 'sigmoid'))

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                480       
_________________________________________________________________
dense_1 (Dense)              (None, 24)                408       
_________________________________________________________________
dropout (Dropout)            (None, 24)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                500       
_________________________________________________________________
dense_3 (Dense)              (None, 24)                504       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 25        
Total params: 1,917
Trainable params: 1,917
Non-trainable params: 0
______________________________________________________

In [10]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


In [11]:
model.fit(X_train, y_train, batch_size = 15, epochs = 5)

Epoch 1/5
13291/13291 [==============================] - 18s 1ms/step - loss: 0.0084 - accuracy: 0.9988
Epoch 2/5
13291/13291 [==============================] - 18s 1ms/step - loss: 0.0040 - accuracy: 0.9992
Epoch 3/5
13291/13291 [==============================] - 18s 1ms/step - loss: 0.0036 - accuracy: 0.9993
Epoch 4/5
13291/13291 [==============================] - 18s 1ms/step - loss: 0.0037 - accuracy: 0.9994
Epoch 5/5
13291/13291 [==============================] - 18s 1ms/step - loss: 0.0036 - accuracy: 0.9993


We achieved an accuracy of 99.94% on trainind dataset without any sampling.

In [12]:
#Evaluate model
score = model.evaluate(X_test, y_test)
print(score)

2671/2671 [==============================] - 2s 923us/step - loss: 0.0036 - accuracy: 0.9994
[0.003607358317822218, 0.9993914365768433]


In [13]:
y_pred = model.predict(X_test)
cm = confusion_matrix(y_test, y_pred.round())
print(cm)

[[85275    21]
 [   31   116]]


In [14]:
print(accuracy_score(y_test, y_pred.round()))
print(precision_score(y_test, y_pred.round()))
print(recall_score(y_test, y_pred.round()))
print(f1_score(y_test, y_pred.round()))

0.9993914071369217
0.8467153284671532
0.7891156462585034
0.8169014084507042


We can see there is a vast difference between all the metrics, so now we can use a sampling method ,SMOTE.

In [15]:
X_resample, y_resample = SMOTE().fit_sample(X, y)
counter = Counter(y_resample)
print(counter)

Counter({0: 284315, 1: 284315})


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X_resample, y_resample, test_size = 0.3)
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [17]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.fit(X_train, y_train, batch_size = 15, epochs = 5)

Epoch 1/5
26537/26537 [==============================] - 39s 1ms/step - loss: 0.0303 - accuracy: 0.9900
Epoch 2/5
26537/26537 [==============================] - 39s 1ms/step - loss: 0.0138 - accuracy: 0.9963
Epoch 3/5
26537/26537 [==============================] - 40s 2ms/step - loss: 0.0116 - accuracy: 0.9971
Epoch 4/5
26537/26537 [==============================] - 44s 2ms/step - loss: 0.0107 - accuracy: 0.9976
Epoch 5/5
26537/26537 [==============================] - 38s 1ms/step - loss: 0.0090 - accuracy: 0.9979


In [19]:
y_pred = model.predict(X_test)
y_expect = pd.DataFrame(y_test)
cm = confusion_matrix(y_expect, y_pred.round())
print(cm)


[[84891   203]
 [  332 85163]]


In [20]:
print(accuracy_score(y_test, y_pred.round()))
print(precision_score(y_test, y_pred.round()))
print(recall_score(y_test, y_pred.round()))
print(f1_score(y_test, y_pred.round()))

0.9968638071622438
0.9976220040765644
0.99611673197263
0.9968687997846202


After re-training the DNN model, SMOTE gives a precision of 99.8%, a recall of 100%, and an F1 score of 99.9%. Much better than DNN .